In [35]:
import torch
from torch.nn import Linear
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, DataLoader
import torch.nn.functional as F
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import pandas as pd
import scipy.special as SS
import scipy.stats as SSA
import copy
import math
from sklearn.model_selection import ParameterGrid
import os
import numpy.linalg as LA
import gzip

# load pickle module
import pickle

The observation is 
3142 x 60 

we can i) treat 3142 nodes and 60 features of each nodes 

(or 60 nodes 3142 features of the each nodes)

In this case the mobility data is the graph structure

In [3]:
class MatrixClassifier(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(MatrixClassifier, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_features)
        self.classifier = Linear(num_features, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Use GCN layers to update node features
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)

        # Use global mean pooling to get a representation of the whole graph
        x = global_mean_pool(x, data.batch)

        # Use a linear layer to classify the graph
        x = self.classifier(x)

        return F.log_softmax(x, dim=1)

In [27]:
# Number of nodes (rows of the matrix)
num_nodes = 4

# Number of features per node (columns of the matrix)
num_features = 2

# Number of classes
num_classes = 2

# Create a random matrix
matrix = torch.randn(num_nodes, num_features)

# Create a fully connected graph
edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j]).t().contiguous()


# Create a random target class
y = torch.randint(num_classes, (1,))

# Create a data object
data = Data(x=matrix, edge_index=edge_index, y=y)

# Create a model and an optimizer
model = MatrixClassifier(num_features, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [31]:
y = torch.tensor([random.randint(0, 1)], dtype=torch.long)

In [33]:
# Create a list to hold our Data objects
dataset = []

# Assume we have 10 graphs with the same structure for this example
for _ in range(10):
    # Each graph is associated with a random target (0 or 1)
    y = torch.tensor([random.randint(0, 1)], dtype=torch.long)
    
    # Create a Data object for each graph
    data = Data(x=matrix, edge_index=edge_index, y=y)
    
    # Add the Data object to our list
    dataset.append(data)

# Now we can create a DataLoader
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [34]:
# 假设 `dataset` 是一个图的列表
# loader = DataLoader(dataset, batch_size=32, shuffle=True)

# 设定训练的epoch数量
num_epochs = 100

# Training loop
# for epoch in range(100):
#     optimizer.zero_grad()
#     out = model(data)
#     loss = F.nll_loss(out, data.y)
#     loss.backward()
#     optimizer.step()

for epoch in range(num_epochs):
    for batch in loader:
        # batch是一个Batch对象，包含了批量图的数据
        batch_data, batch_target = batch.x, batch.y

        # 如果你使用GPU，需要将数据移动到GPU
#         batch_data, batch_target = batch_data.to(device), batch_target.to(device)

        optimizer.zero_grad()  # 清空梯度
        output = model(batch)  # 前向计算
        loss = F.nll_loss(output, batch_target)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数

    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 0.7218454480171204
Epoch: 2, Loss: 0.6985946893692017
Epoch: 3, Loss: 0.6829551458358765
Epoch: 4, Loss: 0.6747191548347473
Epoch: 5, Loss: 0.6731728315353394
Epoch: 6, Loss: 0.6762601733207703
Epoch: 7, Loss: 0.6803207397460938
Epoch: 8, Loss: 0.682461142539978
Epoch: 9, Loss: 0.6821051836013794
Epoch: 10, Loss: 0.6800498366355896
Epoch: 11, Loss: 0.6774061918258667
Epoch: 12, Loss: 0.6750756502151489
Epoch: 13, Loss: 0.6735717058181763
Epoch: 14, Loss: 0.6730198860168457
Epoch: 15, Loss: 0.6732522249221802
Epoch: 16, Loss: 0.6739424467086792
Epoch: 17, Loss: 0.6747397184371948
Epoch: 18, Loss: 0.6753655672073364
Epoch: 19, Loss: 0.6756609082221985
Epoch: 20, Loss: 0.6755892038345337
Epoch: 21, Loss: 0.67521071434021
Epoch: 22, Loss: 0.6746443510055542
Epoch: 23, Loss: 0.6740313768386841
Epoch: 24, Loss: 0.6735018491744995
Epoch: 25, Loss: 0.6731492280960083
Epoch: 26, Loss: 0.6730135679244995
Epoch: 27, Loss: 0.6730775833129883
Epoch: 28, Loss: 0.6732746958732605
Epoc

In [39]:
edge_index.dtype

torch.int64

In [40]:
matrix.dtype

torch.float32

In [41]:
data.y.dtype

torch.int64